# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
import re

# Import necessary libraries for the pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, classification_report,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin

# using word2vec
from gensim.models import Word2Vec

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.data.clear_cache()

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

# Initialize stop words and lemmatizer
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine("sqlite:///DesastrousDatabase.db")
# number of sample for generating model
n = 0
if n != 0:
    df = pd.read_sql_table("categorized_massages", engine).sample(n)
else:
    df = pd.read_sql_table("categorized_massages", engine)
X = df["message"]
Y = df.drop(columns=["id", "message", "original", "genre"])

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Normalize text (lowercase and remove punctuation)
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Lemmatize and remove stop words
    clean_tokens = [lemmatizer.lemmatize(tok).strip() for tok in tokens if tok not in stop_words]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline(
    [
        ("vect", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer()),
        ("clf", MultiOutputClassifier(RandomForestClassifier())),
    ]
)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Test the pipeline on the test data
y_pred = pipeline.predict(X_test)

c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# Convert predictions and true values to DataFrame to match column names
y_pred_df = pd.DataFrame(y_pred, columns=Y.columns)
y_test_df = pd.DataFrame(y_test, columns=Y.columns)

# Iterate through each category and display classification metrics
for column in Y.columns:
    print(f'Category: {column}\n')
    print(classification_report(y_test_df[column], y_pred_df[column]))
    print('-'*60)

Category: related

              precision    recall  f1-score   support

           0       0.69      0.42      0.52      1237
           1       0.84      0.94      0.89      3967
           2       0.59      0.42      0.49        40

    accuracy                           0.81      5244
   macro avg       0.71      0.60      0.63      5244
weighted avg       0.80      0.81      0.80      5244

------------------------------------------------------------
Category: request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4358
           1       0.80      0.49      0.61       886

    accuracy                           0.89      5244
   macro avg       0.85      0.73      0.78      5244
weighted avg       0.89      0.89      0.88      5244

------------------------------------------------------------
Category: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5221
      

c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4982
           1       0.84      0.10      0.18       262

    accuracy                           0.95      5244
   macro avg       0.90      0.55      0.58      5244
weighted avg       0.95      0.95      0.94      5244

------------------------------------------------------------
Category: search_and_rescue

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5115
           1       0.62      0.06      0.11       129

    accuracy                           0.98      5244
   macro avg       0.80      0.53      0.55      5244
weighted avg       0.97      0.98      0.97      5244

------------------------------------------------------------
Category: security

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5156
           1       0.00      0.00      0.00        88

    accurac

c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5214
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      0.99      0.99      5244

------------------------------------------------------------
Category: hospitals

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5175
           1       0.00      0.00      0.00        69

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.97      0.99      0.98      5244

------------------------------------------------------------
Category: shops

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5220
           1       0.00      0.00      0.00        24

    accuracy          

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# Define parameter grid for GridSearchCV
parameters = {
    #'vect__max_df': [0.75, 0.85, 1.0],  # Maximum document frequency for words
    'vect__ngram_range': ((1, 1), (1, 2)),  # Unigrams or unigrams and bigrams
    #'tfidf__use_idf': [True, False],  # Use IDF or not
    #'tfidf__norm': ['l1', 'l2'],  # Normalization method
    'clf__estimator__n_estimators': [50, 100],  # Number of trees in RandomForest
    #'clf__estimator__max_depth': [None, 10, 20],  # Maximum depth of trees
    'clf__estimator__min_samples_split': [2, 3],  # Minimum samples required to split an internal node
    #'clf__estimator__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    #'clf__estimator__bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}


# Initialize GridSearchCV with the pipeline and parameter grid
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=-1)

# Fit the model with GridSearchCV
cv.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
# Predict using the tuned model
y_pred = cv.predict(X_test)

# Convert predictions and true values to DataFrame to match column names
y_pred_df = pd.DataFrame(y_pred, columns=Y.columns)
y_test_df = pd.DataFrame(y_test, columns=Y.columns)


# Iterate through each category and display classification metrics
for column in Y.columns:
    print(f'Category: {column}\n')
    
    # Get the true and predicted values for the current column
    y_true = y_test_df[column]
    y_pred = y_pred_df[column]

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)  # Use 'binary' for binary targets
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    #conf_matrix = confusion_matrix(y_true, y_pred)

    # Print the metrics
    print(f'Accuracy: {accuracy:.2f}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    #print(classification_report(y_test_df[column], y_pred_df[column]))
    #print(f'Confusion Matrix:\n{conf_matrix}\n')
    print('-' * 30)


Category: related

Accuracy: 0.80
Precision: 0.61
Recall: 0.54
------------------------------
Category: request

Accuracy: 0.88
Precision: 0.86
Recall: 0.69
------------------------------
Category: offer

Accuracy: 1.00
Precision: 0.50
Recall: 0.50
------------------------------
Category: aid_related

Accuracy: 0.74
Precision: 0.74
Recall: 0.73
------------------------------
Category: medical_help

Accuracy: 0.92
Precision: 0.80
Recall: 0.51
------------------------------
Category: medical_products

Accuracy: 0.95
Precision: 0.47
Recall: 0.50
------------------------------
Category: search_and_rescue

Accuracy: 0.98
Precision: 0.49
Recall: 0.50
------------------------------
Category: security

Accuracy: 0.98
Precision: 0.49
Recall: 0.50
------------------------------
Category: military

Accuracy: 0.97
Precision: 0.48
Recall: 0.50
------------------------------
Category: child_alone

Accuracy: 1.00
Precision: 1.00
Recall: 1.00
------------------------------
Category: water

Accuracy: 0

As we can see our accuracy is pretty high but the Recall is in some cases low. lets try to get an overall high recall score. 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
# lets add a Starting verb Extractor for a better classification of sentences:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ["VB", "VBP"] or first_word == "RT":
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


# pipeline with featureUnion

pipeline2 = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [
                    (
                        "text_pipeline",
                        Pipeline(
                            [
                                (("vect", CountVectorizer(tokenizer=tokenize))),
                                (("tfidf", TfidfTransformer())),
                            ]
                        ),
                    ),
                    ("starting_verb", StartingVerbExtractor()),
                ]
            ),
        ),
        ("multi_rf", MultiOutputClassifier(RandomForestClassifier())),
    ]
)


# Define parameter grid for GridSearchCV
parameters = {
    "features__text_pipeline__vect__ngram_range": (
        (1, 1),
        (1, 2),
    ),  # Unigrams or unigrams and bigrams
    "features__text_pipeline__vect__max_df": [
        0.5,
        0.75,
        1.0,
    ],  # Maximum document frequency for words
    "features__text_pipeline__vect__max_features": (
        None,
        5000,
        10000,
    ),  # Maximum document frequency for words
    "features__text_pipeline__tfidf__use_idf": [
        True,
        False,
    ],  # Whether to use idf in TfidfTransformer
    # "features__text_pipeline__tfidf__norm": ["l1", "l2"],  # Normalization method
    "multi_rf__estimator__n_estimators": [
        50,
        100,
        200,
    ],  # Number of trees in RandomForest
    # "multi_rf__estimator__max_depth": [None, 10, 20],  # Maximum depth of trees
    "multi_rf__estimator__min_samples_split": [
        2,
        3,
        5,
    ],  # Minimum samples required to split an internal node
    #'multi_rf__estimator__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    "features__transformer_weights": (
        {"text_pipeline": 1, "starting_verb": 0.5},
        {"text_pipeline": 0.5, "starting_verb": 1},
        {"text_pipeline": 0.8, "starting_verb": 1},
    ),
}

# Initialize GridSearchCV with the pipeline and parameter grid
cv2 = GridSearchCV(pipeline2, param_grid=parameters, verbose=3, n_jobs=-1)

# Fit the model with GridSearchCV
cv2.fit(X_train, y_train)

# Predict using the tuned model
y_pred = cv2.predict(X_test)

# Convert predictions and true values to DataFrame to match column names
y_pred_df = pd.DataFrame(y_pred, columns=Y.columns)
y_test_df = pd.DataFrame(y_test, columns=Y.columns)

# Iterate through each category and display classification metrics
for column in Y.columns:
    print(f"Category: {column}\n")

    # Get the true and predicted values for the current column
    y_true = y_test_df[column]
    y_pred = y_pred_df[column]

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(
        y_true, y_pred, average="macro", zero_division=0
    )  # Use 'binary' for binary targets
    recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
    # conf_matrix = confusion_matrix(y_true, y_pred)

    # Print the metrics
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    # print(f'Confusion Matrix:\n{conf_matrix}\n')
    print("-" * 30)

Fitting 5 folds for each of 972 candidates, totalling 4860 fits


KeyboardInterrupt: 

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.